## Piston simulation function

The Piston Simulation function models the circular motion of a piston within a cylinder. It involves a chain of nonlinear functions, see [here](https://www.sfu.ca/~ssurjano/piston.html) for the equations.

The response C is cycle time (the time it takes to complete one cycle), in seconds. 

The input variables and their usual input ranges are:

* M in [30, 60] 	piston weight (kg)
* S in [0.005, 0.020] 	piston surface area (m2)
* V0 in [0.002, 0.010] 	initial gas volume (m3)
* k in [1000, 5000] 	spring coefficient (N/m)
* P0 in [90000, 110000]    	atmospheric pressure (N/m2)
* Ta in [290, 296]    	ambient temperature (K)
* T0 in [340, 360]    	filling gas temperature (K)


In [120]:
#this function implements the equations
import numpy as np

def piston(xx):

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%
    #% OUTPUT AND INPUT:
    #%
    #% C = cycle time
    #% xx = [M, S, V0, k, P0, Ta, T0]
    #%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    M  = xx[0];
    S  = xx[1];
    V0 = xx[2];
    k  = xx[3];
    P0 = xx[4];
    Ta = xx[5];
    T0 = xx[6];

    Aterm1 = P0 * S;
    Aterm2 = 19.62 * M;
    Aterm3 = -k*V0 / S;
    A = Aterm1 + Aterm2 + Aterm3;

    Vfact1 = S / (2*k);
    Vfact2 = np.sqrt(A**2 + 4*k*(P0*V0/T0)*Ta);
    V = Vfact1 * (Vfact2 - A);

    fact1 = M;
    fact2 = k + (S**2)*(P0*V0/T0)*(Ta/(V**2));

    C = 2 * np.pi * np.sqrt(fact1/fact2);

    return C


# We make a dataset
n=200
M=np.random.uniform(30, 60, n)[:,None] #piston weight (kg)
S=np.random.uniform(0.005, 0.020, n)[:,None] #piston surface area (m2)
V0=np.random.uniform(0.002, 0.010, n)[:,None]  #initial gas volume (m3)
k=np.random.uniform(1000, 5000, n)[:,None] #spring coefficient (N/m)
P0=np.random.uniform(90000, 110000, n)[:,None] #atmospheric pressure (N/m2)
Ta=np.random.uniform(290, 296, n)[:,None] #ambient temperature (K)
T0=np.random.uniform(340, 360, n)[:,None] #filling gas temperature (K)

np.random.seed(42)
X=np.hstack([M,S,V0,k,P0,Ta,T0])#dataset
Y=[]
for i in range(n):
    Y.append(piston(X[i,:])+np.random.randn(1)*0.01)#we add some Gaussian noise
Y=np.array(Y)

Your **goal** is to use a Bayesian NN to learn a surrogated model of the Piston function and compute the posterior of the prediction of the output $C$ at $Xpred=[45,0.015,0.008,3500,100000,290,341]$ and compare it with:
* the true value 
* the prediction returned by Keras for the same NN structure.

In [118]:
Xtest=np.array([[45,0.015,0.008,3500,100000,290,341]])
piston(Xtest[0,:])

0.4603480963134951